In [1]:
import cv2
import numpy as np
import face_recognition
import os


images = []
classNames = []

# # Reading the training images and classes and storing into the corresponsing lists
# for img in os.listdir(path):
#     image = cv2.imread(f'{path}/{img}')
#     images.append(image)
#     classNames.append(os.path.splitext(img)[0])


image = cv2.imread("Manmeet.jpg")
images.append(image)
classNames.append(os.path.splitext("Manmeet.jpg")[0])
image = cv2.imread("Apoorv.jpeg")
images.append(image)
classNames.append(os.path.splitext("Apoorv.jpeg")[0])
image = cv2.imread("virat.jpeg")
images.append(image)
classNames.append(os.path.splitext("virat.jpeg")[0])

print(classNames)
images



['Manmeet', 'Apoorv', 'virat']


[array([[[126, 135, 144],
         [126, 136, 143],
         [129, 137, 144],
         ...,
         [ 64,  23,  28],
         [ 62,  19,  22],
         [ 65,  20,  23]],
 
        [[122, 131, 140],
         [123, 133, 140],
         [126, 135, 139],
         ...,
         [ 62,  22,  27],
         [ 62,  19,  22],
         [ 63,  20,  23]],
 
        [[122, 130, 137],
         [122, 130, 137],
         [123, 132, 136],
         ...,
         [ 61,  21,  26],
         [ 60,  20,  22],
         [ 61,  21,  23]],
 
        ...,
 
        [[116, 113, 115],
         [117, 114, 116],
         [121, 116, 118],
         ...,
         [ 31,  11,  16],
         [ 32,  11,  14],
         [ 32,  11,  14]],
 
        [[118, 115, 117],
         [120, 117, 119],
         [124, 119, 121],
         ...,
         [ 30,  10,  15],
         [ 32,  11,  14],
         [ 31,  10,  13]],
 
        [[123, 120, 122],
         [126, 123, 125],
         [128, 123, 125],
         ...,
         [ 30,  10,  15],
  

In [2]:
scale = 0.25
box_multiplier = 1/scale


# Function for Find the encoded data of the imput image
def findEncodings(images):
    encodeList = []
    for img in images:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        encode = face_recognition.face_encodings(img)[0]
        print(encode)
        encodeList.append(encode)
    return encodeList

# Find encodings of training images

knownEncodes = findEncodings(images)
print('Encoding Complete')

[-9.78395268e-02  8.67507383e-02 -1.82559080e-02  1.40339127e-02
 -3.86578664e-02 -2.05066707e-02 -2.45155040e-02 -3.77994180e-02
  1.64991155e-01 -8.00098330e-02  1.98905081e-01 -5.06183058e-02
 -2.26031914e-01 -3.74882948e-05 -7.46410713e-02  6.54174164e-02
 -1.65644437e-01 -5.79086430e-02 -2.62573045e-02 -7.94406682e-02
  2.84605920e-02  3.47853824e-02  3.17350253e-02  4.59046774e-02
 -8.87907147e-02 -3.52311820e-01 -8.48122239e-02 -1.41170576e-01
  2.16692351e-02 -4.64900769e-02 -3.58330086e-02  2.22399049e-02
 -1.51587814e-01 -5.15837520e-02 -8.86176713e-03  3.58575694e-02
 -6.73344508e-02  4.30222638e-02  2.12616026e-01  2.07482092e-02
 -9.79532078e-02 -5.31330556e-02  6.25173226e-02  3.28999728e-01
  1.65148929e-01  8.77956375e-02 -4.13862057e-03 -8.01538676e-02
  1.26702532e-01 -2.14622498e-01  1.27232417e-01  1.49073914e-01
  1.35873362e-01  1.03921287e-01  7.54984692e-02 -1.19864151e-01
 -7.41372816e-03  8.12015384e-02 -1.43708453e-01  1.01767875e-01
  4.83721308e-02 -9.91277

In [ ]:
cap = cv2.VideoCapture(0)
 
while True:
    success, img = cap.read()  # Reading Each frame
    
    # Resize the frame
    Current_image = cv2.resize(img,(0,0),None,scale,scale)
    Current_image = cv2.cvtColor(Current_image, cv2.COLOR_BGR2RGB)

    # Find the face location and encodings for the current frame
    # 'cnn' runs on gpu and it is more accurate. change it to 'hog' is you want to run on cpu.
    face_locations = face_recognition.face_locations(Current_image, model='cnn')  
    face_encodes = face_recognition.face_encodings(Current_image,face_locations)

    # print(face_locations)
    
    # Find the matches for each detection
    
    for encodeFace,faceLocation in zip(face_encodes,face_locations):
        matches = face_recognition.compare_faces(knownEncodes,encodeFace, tolerance=0.6)
        # print(matches)
        faceDis = face_recognition.face_distance(knownEncodes,encodeFace)
        matchIndex = np.argmin(faceDis)

        # print(matchIndex)
        # If match found then get the class name for the corresponding match

        if matches[matchIndex]:
            name = classNames[matchIndex].upper()

        else:
            name = 'Unknown'

        y1,x2,y2,x1 = faceLocation
        y1, x2, y2, x1 = int(y1*box_multiplier),int(x2*box_multiplier),int(y2*box_multiplier),int(x1*box_multiplier)

        # Draw rectangle around detected face

        cv2.rectangle(img,(x1,y1),(x2,y2),(0,255,0),2)
        cv2.rectangle(img,(x1,y2-20),(x2,y2),(0,255,0),cv2.FILLED)
        cv2.putText(img,name,(x1+6,y2-6),cv2.FONT_HERSHEY_COMPLEX,0.5,(255,255,255),2)


    # show the output
    cv2.imshow('Webcam',img)

    if cv2.waitKey(1) == ord('q'):
        break

# release the camera object

cap.release()
cv2.destroyAllWindows()


In [ ]:
# release the camera object

cap.release()
cv2.destroyAllWindows()